In [25]:
pip install tweepy


Note: you may need to restart the kernel to use updated packages.


In [48]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [49]:
from textblob import TextBlob
import re

In [1]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import Cursor
from tweepy import API

In [36]:
import  numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [37]:
CONSUMER_KEY=  'IeWOmiS4IVadArPPAVHLDFJbF'
API_SECRET='w2Nvll3a23jBbOPLmbTIYn5txkcRrqYv1QeD73JCkEOAHmye1h'
ACCESS_TOKEN= '1336281671251816449-cEOGUINpPsLrBWnm86Q7PyAAQDKyU5'
CONSUMER_SECRET_KEY= 'e37aNcAU9ze9zWFCFcrRT8JgZ4Y13GOfSTeTpbWYIaB9U'

In [38]:
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user
    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets
    def get_friend_list(self,num_friends):
        friend_list= []
        for friend in Cursor(self.twitter_clint.friends,id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return(friend_list)
    def get_home_timeline_tweets(self,num_tweets):
        home_timeline_tweets=[]
        for tweet in Cursor(self.twitter_clint.home_timeline,id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets
        
        
        
        
class TwitterAuthenticator():
    def authenticate_twitter_app(self):
        auth=OAuthHandler(CONSUMER_KEY,API_SECRET)
        auth.set_access_token(ACCESS_TOKEN,CONSUMER_SECRET_KEY)
        return auth
         
        
class TwitterStreamer():
    def __init__(self):
        self.twitter_authenticator=TwitterAuthenticator()
    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        listener=TwitterListener()
        auth=self.twitter_authenticator.auithenticate_twitter_app()
        stream=Stream(auth,listener)
        stream.filter(track=hash_tag_list)

    
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)

 


In [67]:
class TweetAnalyzer():
    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def analyze_sentiment(self,tweet):
        analysis=TextBlob(self.clean_tweet(tweet))
        if analysis.sentiment.polarity>0:
            return 1
        elif analysis.sentiment.polarity==0:
            return 0
        else:
            return -1
        
    def tweets_to_data_frame(self,tweets):
        df=pd.DataFrame(data=[tweet.text for tweet in tweets],columns=['tweets'])
        df['id']=np.array([tweet.id for tweet in tweets])
        df['len']=np.array([len(tweet.text) for tweet in tweets])
        df['date']=np.array([tweet.created_at for tweet in tweets])
        df['source']=np.array([tweet.source for tweet in tweets])
        df['likes']=np.array([tweet.favorite_count for tweet in tweets])
        df['retweets']=np.array([tweet.retweet_count for tweet in tweets])
        return df

In [69]:
if __name__ =='__main__':
    twitter_client=TwitterClient()
    tweet_analyzer=TweetAnalyzer()
    api=twitter_client.get_twitter_client_api()
    
    tweets=api.user_timeline(screen_name="ArvindKejriwal",count=20)
    df=tweet_analyzer.tweets_to_data_frame(tweets)
    #Get average lengeth pver all tweets:
    print(np.mean(df['len']))
    #print(df.head())
    print(np.max(df['retweets']))
    #time_likes=pd.Series(data=df['likes'].values,index=df['date'])
    #time_likes.plot(figsize=(16,4),color='r')
    #plt.show()
    time_retweets=pd.Series(data=df['retweets'].values,index=df['date'])
    #time_retweets.plot(figsize=(16,4),color='r')
    #plt.show()
        
    
    df['sentiment']=np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])
    
    print(df.head(20))
            
    

135.7
6676
                                               tweets                   id  \
0   I strongly condemn the attack on Mamta Didi. T...  1369664513184722947   
1   प्रभु श्री राम का मानना था कि बिना किसी भेदभाव...  1369632451329368069   
2   पिछले 70 साल में दोनों राजनीतिक पार्टियों ने म...  1369632402033713153   
3   उपराज्यपाल महोदय के अभिभाषण के लिए धन्यवाद प्र...  1369559774803595265   
4   स्त्री शिक्षणासाठी आपले जीवन समर्पित करणाऱ्या ...  1369504579054886912   
5   दिल्ली सरकार ने शहीद सिपाहियों के परिवार के सम...  1369239240580730880   
6   दिल्ली सरकार ने आज़ादी के 75वें वर्ष में शहीद ...  1369239195320000520   
7   आज़ादी की 75वीं वर्षगांठ के अवसर पर आगामी साल ...  1369239160872079360   
8   दिल्ली सरकार का लक्ष्य है कि वर्ष 2047 तक दिल्...  1369238982987497475   
9   हमें शिक्षा को जन आंदोलन बनाने की जरूरत है। ' ...  1369238483840233473   
10  पूरी दिल्ली में दिल्ली सरकार 500 स्थानों पर शा...  1369238392009936896   
11  दिल्ली की प्रगति के लिए इस साल का ये बजट उन्नत...